In [2]:
import requests
import json
import numpy as np

In [3]:
PREDICT_URL = "http://127.0.0.1:8000/predict/"
RESULT_URL = "http://127.0.0.1:8000/result/"
RETRAIN_URL = "http://127.0.0.1:8000/retrain/"
NEWMODEL_URL = "http://127.0.0.1:8000/new_model/"
EVALUATE_URL = "http://127.0.0.1:8000/evaluate_model/"

DUMMY_MODEL = "Dummy"
KNN_MODEL = "KNN"

metric = 0.0
key = 0

In [4]:
def send_get_request (url:str, params:dict) :

    data = {}
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        print(data)
    else:
        print(f"Error: {response.status_code}")
    return data

In [5]:
def get_prediction(key, x, y) :
    params = {"id" : key, "x" :x, "y" :y}
    response = send_get_request(PREDICT_URL, params)
    return response.get("prediction")

In [6]:
def send_result(key, result) :
    params = {"id" : key, "result" : result}
    response = send_get_request(RESULT_URL, params)
    return response

In [7]:
def request_retrain(last_n:int) :
    params = {"last_n" : last_n}
    response = send_get_request(RETRAIN_URL, params)
    return response

In [8]:
def request_new_model(model_name:str, model_params:dict, last_n:int) :
    params = {"last_n" : last_n, "model": model_name, "model_params":model_params}
    response = send_get_request(NEWMODEL_URL)
    return response

In [9]:
def request_evaluation(last_n:int) :
    params = {"last_n" : last_n}
    response = send_get_request(EVALUATE_URL, params)
    return response

In [10]:
def new_dummy(strategy: str, last_n: int) :
    model_params = {"strategy" : strategy}
    response = request_new_model(DUMMY_MODEL, model_params, last_n)
    return response

In [11]:
def new_knn(k:int, last_n: int):
    model_params = {"k" : k}
    response = request_new_model(KNN_MODEL, model_params, last_n)
    return response

In [12]:
def item(key,metric) :
    key=key+1
    x=np.random.rand()*2-1
    y=np.random.rand()*2-1
    z=np.random.rand()*.2-.1
    result=x*y+z>metric
    return [key, x, y, result]

In [3]:
import numpy as np

def item(key, metric):
    key = key + 1
    x = np.random.rand() * 2 - 1
    y = np.random.rand() * 2 - 1
    z = np.random.rand() * .2 - .1
    result = x * y + z > metric
    return [key, x, y, result]

key = 0  # Initialize key
metric = 0.0  # Initialize metric

key, x, y, result = item(key, metric)

In [ ]:


# FastAPI endpoint definition (example)
# @app.post("/items/")
# async def create_item(item: Item):
#     return item

#url = "http://127.0.0.1:8000/items/"  # Replace with your actual URL
#data = {"name": "New Item", "description": "A new item description"}
#headers = {'Content-type': 'application/json'}
#response = requests.post(url, data=json.dumps(data), headers=headers)

#if response.status_code == 200:
#    data = response.json()
#    print(data)
#else:
#    print(f"Error: {response.status_code}")

MY CODES_

In [16]:
pip install fastapi

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 4.6 MB/s eta 0:00:00


In [17]:
pip install uvicorn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 2.1 MB/s eta 0:00:00


In [6]:
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.dummy import DummyClassifier
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, recall_score, precision_score
import fastapi
from typing import Optional

In [7]:
# Initialize sample training data: one x-value
_x = [1]  # x-coordinate of one data point
_y = [1]  # y-coordinate of the same data point

# Prediction and result placeholders for the same data point
_prediction = ["True"]  # what the model predicted
_result = ["True"]      # what the actual result was (ground truth)

# Create a DummyClassifier that always predicts the same value ("True")
model = DummyClassifier(strategy="constant", constant="True")

# Fit the model on the single data point [1, 1] with the result "True"
# Reshape is used to make the data into the shape (1, 2) as required by scikit-learn
model.fit(np.array([_x, _y]).reshape(1, -1), _result)


DummyClassifier(constant='True', strategy='constant')

In [8]:
def model_evaluation(y_test, y_pred):
    # Print the accuracy score as a percentage for easier human reading
    print('Trained Model Test Data Accuracy Score :', accuracy_score(y_test, y_pred) * 100)

    # Calculate raw accuracy score (between 0 and 1)
    testacc = accuracy_score(y_test, y_pred)

    # Calculate recall for the positive class ('True')
    # Recall = TP / (TP + FN)
    testrecall = recall_score(y_test, y_pred, pos_label='True')

    # Calculate precision for the positive class ('True')
    # Precision = TP / (TP + FP)
    testprecision = precision_score(y_test, y_pred, pos_label='True')

    print(' ')  # Print a blank line for spacing

    # Print a detailed classification report including precision, recall, f1-score, and support
    print(classification_report(y_test, y_pred))

    # Return the three key evaluation metrics for further use
    return testacc, testrecall, testprecision


In [9]:
def create_eval_df(x, y, result):
    # Step 1: Create a dictionary of input features and results
    data = {"x": x, "y": y, "result": result}

    # Step 2: Convert the dictionary into a pandas DataFrame
    df = pd.DataFrame(data)

    # Step 3: Make a copy of the DataFrame for evaluation purposes
    eval_df = df.copy()

    # Step 4: Predict outcomes using the current model on the input features
    # Note: data[x, y] is not the correct way to index a dictionary. It should be:
    # model.predict(np.array([x, y]).T)
    eval_df["predict"] = model.predict(np.array([x, y]).T)

    # Step 5: Add a boolean column showing whether each prediction was correct
    eval_df["correct"] = eval_df["predict"] == eval_df["result"]

    # Step 6: Add a column showing if both prediction and result were "True"
    eval_df["correctandtrue"] = eval_df["predict"].isin(["True"]) & eval_df["result"].isin(["True"])

    # Step 7: Return the final DataFrame with all evaluations
    return eval_df


In [12]:
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.metrics import accuracy_score, recall_score, precision_score
import warnings
import fastapi # import fastapi

app = fastapi.FastAPI() # create a FastAPI instance

@app.get("/retrain/")
def retrain(last_n: int, stratify: bool = True):
    global model
    print(f"RETRAINING over last {last_n}")

    n = min(last_n, len(_x) - 1)
    X = np.array([_x[-n:], _y[-n:]]).T
    Y = np.array(_result[-n:])

    # Train-test split with or without stratification
    if stratify and len(np.unique(Y)) > 1:
        sss = StratifiedShuffleSplit(n_splits=1, test_size=0.4, random_state=42)
        for train_index, test_index in sss.split(X, Y):
            X_train, X_test = X[train_index], X[test_index]
            y_train, y_test = Y[train_index], Y[test_index]
    else:
        X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.4, random_state=42)

    # Verbose display
    print(X_train)
    print(y_train)
    print(X_test)
    print(y_test)

    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    print("Trained Model Test Data Accuracy Score :", accuracy_score(y_test, y_pred) * 100)
    print(" ")
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        from sklearn.metrics import classification_report
        print(classification_report(y_test, y_pred))

    # Handle metrics with fallback for 0-div
    acc = accuracy_score(y_test, y_pred)
    # Change pos_label="True" to pos_label=True
    rec = recall_score(y_test, y_pred, pos_label=True, zero_division=0)
    # Change pos_label="True" to pos_label=True
    prec = precision_score(y_test, y_pred, pos_label=True, zero_division=0)

    return {
        "message": "Model retrained",
        "accuracy": acc,
        "recall": rec,
        "precision": prec
    }


In [13]:
@app.get("/new_model/")
def new_model(model_type: str, last_n: int, strategy: Optional[str] = None, k: Optional[int] = 3, stratify: bool = True):
    global model
    if model_type == "Dummy":
        model = DummyClassifier(strategy=strategy or "most_frequent")
    elif model_type == "KNN":
        model = KNeighborsClassifier(n_neighbors=k)
    else:
        return {"error": "Invalid model_type. Use 'Dummy' or 'KNN'"}

    retrain_result = retrain(last_n=last_n, stratify=stratify)

    return {
        "message": f"Model trained {model}",
        "train": retrain_result
    }



In [14]:
#print(predict(1, 1, 1))
#result(1, True)
# Populate with synthetic data (add this before calling new_model)
for i in range(50):
    _x.append(i)
    _y.append(i % 2)
    _result.append(i % 3 == 0)
new_model("Dummy", 50, strategy="stratified") # Change "stratified" to strategy="stratified"

RETRAINING over last 50
[[37  1]
 [ 2  0]
 [13  1]
 [ 1  1]
 [24  0]
 [ 3  1]
 [ 7  1]
 [14  0]
 [44  0]
 [ 8  0]
 [21  1]
 [15  1]
 [ 9  1]
 [47  1]
 [12  0]
 [40  0]
 [ 0  0]
 [32  0]
 [25  1]
 [29  1]
 [17  1]
 [ 5  1]
 [23  1]
 [48  0]
 [19  1]
 [ 4  0]
 [20  0]
 [36  0]
 [30  0]
 [26  0]]
[False False False False  True  True False False False False  True  True
  True False  True False  True False False False False False False  True
 False False False  True  True False]
[[46  0]
 [41  1]
 [38  0]
 [39  1]
 [45  1]
 [27  1]
 [16  0]
 [42  0]
 [28  0]
 [43  1]
 [49  1]
 [10  0]
 [31  1]
 [ 6  0]
 [34  0]
 [18  0]
 [11  1]
 [35  1]
 [22  0]
 [33  1]]
[False False False  True  True  True False  True False False False False
 False  True False  True False False False  True]
Trained Model Test Data Accuracy Score : 75.0
 
              precision    recall  f1-score   support

       False       0.79      0.85      0.81        13
        True       0.67      0.57      0.62         7

    a

{'message': "Model trained DummyClassifier(strategy='stratified')",
 'train': {'message': 'Model retrained',
  'accuracy': 0.75,
  'recall': 0.5714285714285714,
  'precision': 0.6666666666666666}}

In [16]:
# Define a GET endpoint at /items/
@app.get("/items/")
# Define an asynchronous function that takes two query parameters: skip and limit
async def read_item(skip: int = 0, limit: int = 10):
    # Return the values of skip and limit as a JSON response
    return {"skip": skip, "limit": limit}


In [17]:
# Import the asyncio module for handling asynchronous operations
import asyncio

# Import uvicorn, the ASGI server used to run FastAPI applications
import uvicorn

# Ensure this block runs only when the script is executed directly (not imported)
if __name__ == "__main__":

    # Step 1: Create a Uvicorn configuration using the FastAPI app instance `app`
    config = uvicorn.Config(app)

    # Step 2: Create a Uvicorn server using the above configuration
    server = uvicorn.Server(config)

    # Step 3: Get the currently running asyncio event loop
    loop = asyncio.get_running_loop()

    # Step 4: Schedule the Uvicorn server to run asynchronously within the loop
    loop.create_task(server.serve())
